In [ ]:
import duckdb
from pandas import get_dummies
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

## Get Data


In [ ]:
con = duckdb.connect('my-db.duckdb')
df = con.execute("SELECT * FROM penguins").fetchdf().dropna()
con.close()

df.head(3)

## Define Model and Fit


In [ ]:
X = get_dummies(df[['bill_length_mm', 'species', 'sex']], drop_first = True)
y = df['body_mass_g']

model = LinearRegression().fit(X, y)

## Convert to a Vetiver Model


In [ ]:
from vetiver import VetiverModel
V = VetiverModel(model, model_name = "penguin_model", prototype_data = X)

## Save to Board


In [ ]:
import pins
from vetiver import vetiver_pin_write

# Create or specify a board to pin your model
board = pins.board_folder('board', allow_pickle_read=True)

# Assuming you want to write the model to a pin
vetiver_pin_write(board, V)

## Turn Model Into API


In [ ]:
from vetiver import VetiverAPI
app = VetiverAPI(V, check_prototype = True)

## Prepare Docker


In [ ]:
from vetiver import prepare_docker
prepare_docker(model_board, "penguin_model")

## Get some information


In [ ]:
print(f"R^2 {model.score(X,y)}")
print(f"Intercept {model.intercept_}")
print(f"Columns {X.columns}")
print(f"Coefficients {model.coef_}")